Some questions we might ask are: 
1. What were the fastest and slowest growing population centers in the US? 
2. 

Pop_change notes: 
CHANGE EXPRESSED AS PERCENTAGE (0-100).


In [10]:
import pandas as pd
import numpy as np

pop_change_df = pd.read_csv('data/pop_change.csv', index_col=0)
pop_density_df = pd.read_csv('data/pop_density.csv')

pop_change_df.head(65)

,1910_POPULATION,1920_POPULATION,1930_POPULATION,1940_POPULATION,1950_POPULATION,1960_POPULATION,1970_POPULATION,1980_POPULATION,1990_POPULATION,2000_POPULATION,...,1920_CHANGE,1930_CHANGE,1940_CHANGE,1950_CHANGE,1960_CHANGE,1970_CHANGE,1980_CHANGE,1990_CHANGE,2000_CHANGE,2010_CHANGE
STATE_OR_REGION,,,,,,,,,,,,,,,,,,,,,
United States,92228531,106021568,123202660,132165129,151325798,179323175,203211926,226545805,248709873,281421906,...,15.0,16.2,7.3,14.5,18.5,13.3,11.5,9.8,13.2,9.7
Northeast,25868573,29662053,34427091,35976777,39477986,44677819,49040703,49135283,50809229,53594378,...,14.7,16.1,4.5,9.7,13.2,9.8,0.2,3.4,5.5,3.2
Midwest,29888542,34019792,38594100,40143332,44460762,51619139,56571663,58865670,59668632,64392776,...,13.8,13.4,4.0,10.8,16.1,9.6,4.1,1.4,7.9,3.9
South,29389330,33125803,37857633,41665901,47197088,54973113,62795367,75372362,85445930,100236820,...,12.7,14.3,10.1,13.3,16.5,14.2,20.0,13.4,17.3,14.3
West,7082086,9213920,12323836,14379119,20189962,28053104,34804193,43172490,52786082,63197932,...,30.1,33.8,16.7,40.4,38.9,24.1,24.0,22.3,19.7,13.8
Alabama,2138093,2348174,2646248,2832961,3061743,3266740,3444165,3893888,4040587,4447100,...,9.8,12.7,7.1,8.1,6.7,5.4,13.1,3.8,10.1,7.5
Alaska,64356,55036,59278,72524,128643,226167,300382,401851,550043,626932,...,-14.5,7.7,22.3,77.4,75.8,32.8,33.8,36.9,14.0,13.3
Arizona,204354,334162,435573,499261,749587,1302161,1770900,2718215,3665228,5130632,...,63.5,30.3,14.6,50.1,73.7,36.0,53.5,34.8,40.0,24.6
Arkansas,1574449,1752204,1854482,1949387,1909511,1786272,1923295,2286435,2350725,2673400,...,11.3,5.8,5.1,-2.0,-6.5,7.7,18.9,2.8,13.7,9.1
